In [1]:
import pandas as pd 
import os
import csv
from datetime import datetime, timedelta
import openpyxl

In [2]:
all_files = os.listdir()    
csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))
dfs = []

for csv_file in csv_files:
    print(csv_file)
    dfs.append(pd.read_csv(csv_file))

len(dfs)

camping_par1.csv
camping_par2.csv
camping_par3.csv
camping_par4.csv
camping_par5.csv
camping_par6.csv


6

In [3]:
df = pd.concat(dfs)

In [4]:
len(df)

145613

In [5]:
df.drop(columns=['url'], inplace=True)

In [6]:
dates = pd.bdate_range(start='22-04-2024', end='30-10-2024', freq='C', weekmask='Sat Sun').strftime('%d/%m/%Y').to_list()

In [7]:
dates

['27/04/2024',
 '28/04/2024',
 '04/05/2024',
 '05/05/2024',
 '11/05/2024',
 '12/05/2024',
 '18/05/2024',
 '19/05/2024',
 '25/05/2024',
 '26/05/2024',
 '01/06/2024',
 '02/06/2024',
 '08/06/2024',
 '09/06/2024',
 '15/06/2024',
 '16/06/2024',
 '22/06/2024',
 '23/06/2024',
 '29/06/2024',
 '30/06/2024',
 '06/07/2024',
 '07/07/2024',
 '13/07/2024',
 '14/07/2024',
 '20/07/2024',
 '21/07/2024',
 '27/07/2024',
 '28/07/2024',
 '03/08/2024',
 '04/08/2024',
 '10/08/2024',
 '11/08/2024',
 '17/08/2024',
 '18/08/2024',
 '24/08/2024',
 '25/08/2024',
 '31/08/2024',
 '01/09/2024',
 '07/09/2024',
 '08/09/2024',
 '14/09/2024',
 '15/09/2024',
 '21/09/2024',
 '22/09/2024',
 '28/09/2024',
 '29/09/2024',
 '05/10/2024',
 '06/10/2024',
 '12/10/2024',
 '13/10/2024',
 '19/10/2024',
 '20/10/2024',
 '26/10/2024',
 '27/10/2024']

In [8]:
len(df[df['date_debut'].isin(dates)])

145613

In [9]:
df = df[df['date_debut'].isin(dates)]

In [52]:
# df['date_price'] = "/03/2024"

In [11]:
df.iloc[3370]['prix_init']

805.0

In [10]:
df['prix_init'] = df['prix_init'].apply(lambda x: str(x).replace('\u202f', ''))
df['prix_actuel'] = df['prix_actuel'].apply(lambda x: str(x).replace('\u202f', ''))

In [11]:
df.columns

Index(['web-scrapper-order', 'date_price', 'date_debut', 'date_fin',
       'prix_init', 'prix_actuel', 'typologie', 'n_offre', 'nom',
       'Nb personnes', 'localite', 'date_debut-jour', 'Nb semaines',
       'cle_station', 'nom_station'],
      dtype='object')

In [12]:
df.to_csv("camping_alls.csv", index=False)

In [42]:
items = []
for x in range(len(df)):
    items.append(df.iloc[x].to_dict())
clean_items = []

In [5]:
for res in items:
    if (res['prix_init'] == "") or (res['prix_init'] != res['prix_init']):
        continue
    if (res['prix_actuel'] == "") or (res['prix_actuel'] != res['prix_actuel']):
        continue
    if res['nom'] != res['nom'] or res['nom'] == "":
        continue
    if res['Nb personnes'] != res['Nb personnes'] or res['Nb personnes'] == "":
        continue
    if res['localite'] != res['localite'] or res['localite'] == "":
        continue
    clean_items.append(res)

In [6]:
df_new = pd.DataFrame(clean_items)
df_new.to_csv("camping_all.csv", index=False)

In [13]:
def clean_campings(src, dest):
    def clean_line(line, references):
        for key in line.keys():
            if str(line[key]) == 'nan':
                line[key] = ''

            line['localite'] = line['localite'].split(',')[0].strip()
            if line['localite'] in references.keys() and type(references[line['localite']]['station']) == str:
                line['nom_station'] = references[line['localite']]['station']

            if type(line['nom_station']) != str:
                line['nom_station'] = ""

            line['web-scrapper-order'] = ""
            line['n_offre'] = ""
            line['date_debut-jour'] = ""
            line['localite'] = line['localite'].replace(', ', ' ') if not line['localite'].split(' ')[0].isdigit() else ' '.join(line['localite'].split(' ')[1:]).replace(', ', ' ')
            line['typologie'] = line['typologie'].replace(', ', ' - ')
            line['prix_init'] = str(int(float(line['prix_init']))) if line['prix_init'] != 'prix_init' else 'prix_init'
            line['prix_actuel'] = str(int(float(line['prix_actuel']))) if line['prix_actuel'] != 'prix_actuel' else 'prix_actuel'

        return line
    


    csv_source = pd.read_csv(src, encoding='utf-8')
    csv_source.dropna(subset=['Nb personnes'], inplace=True)
    csv_source = csv_source[csv_source['Nb personnes'].isin(['4 Adultes','6 Adultes', '8 Adultes'])]
    csv_source.drop_duplicates(inplace=True, subset=['date_price', 'date_debut', 'date_fin', 'prix_init', 'prix_actuel', 'typologie', 'nom', 'localite'])
    csv_source.sort_values(inplace=True, ascending = True, by=['Nb semaines', 'date_debut'])
    df_todrop = list(csv_source[csv_source['nom'].isna()].index)
    csv_source.drop(index=df_todrop, inplace=True)
    df_todrop_ty = list(csv_source[csv_source['localite'].isna()].index)
    csv_source.drop(index=df_todrop_ty, inplace=True)
    df_todrop_tz = list(csv_source[csv_source['Nb personnes'].isna()].index)
    print(len(df_todrop))
    csv_source.drop(index=df_todrop_tz, inplace=True)
    csv_dict = csv_source.to_dict(orient='records')

    new_dict = []

    all_references_list = pd.read_excel('referencement stations.xlsm', sheet_name='Feuil1').to_dict(orient='records')

    all_references_dict = {}
    for ref in all_references_list:
        if ref['Localite'] not in all_references_dict.keys():
            all_references_dict[ref['Localite'].split(',')[0]] = {'station': ref['Station'], 'dest': ref['Cle station']}

    for line in csv_dict:
        cleaned_line = clean_line(line, all_references_dict)
        new_dict.append(cleaned_line)

    with open(dest, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['web-scrapper-order', 'date_price', 'date_debut', 'date_fin', 'prix_init', 'prix_actuel', 'typologie', 'Nb personnes','n_offre', 'nom', 'localite', 'date_debut-jour','Nb semaines', 'cle_station', 'nom_station']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for line in new_dict:
            try:
                writer.writerow(line)
            except Exception as e:
                print(e)
                pass

In [15]:
# here x should be replaced with de date of monday of the week scrap
clean_campings('./camping_alls.csv', 'camping_cleaned_29_04_2024.csv')

1324


In [12]:
df = pd.read_csv('./camping_cleaned_12_02_2024.csv', low_memory=False)

In [13]:
nom = df[df['nom'].isna()].index

In [15]:
len(nom)

610